In [ ]:
import click
import pystow
import rdflib
import sssom
from curies.dataframe import filter_df_by_prefixes
from tabulate import tabulate
from biomappings.resources import POSITIVES_SSSOM_PATH
import bioregistry
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
path = pystow.join("nfdi", name="chem-culture.sssom.ttl")
msdf = sssom.parse_sssom_table(POSITIVES_SSSOM_PATH, prefix_map=bioregistry.get_default_converter())
# only keep iconclass mappings
msdf.df = filter_df_by_prefixes(msdf.df, column="subject_id", prefixes=["iconclass"])
sssom.write_rdf(msdf, path, hydrate=True)

graph = rdflib.Graph()
graph.parse(path)

print(graph)

/Users/cthoyt/dev/sssom-py/src/sssom/util.py:184: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("", np.nan, inplace=True)


[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


In [6]:
from rdflib import SKOS
for triple in graph.subject_objects(SKOS.relatedMatch):
    print(triple)

(rdflib.term.URIRef('https://iconclass.org/24C31'), rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_29287'))
(rdflib.term.URIRef('https://iconclass.org/11H%28ANTONY%20ABBOT%29411'), rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_29287'))
(rdflib.term.URIRef('https://iconclass.org/22C4%28GOLD%29'), rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_29287'))
(rdflib.term.URIRef('https://iconclass.org/25D13%28GOLD%29'), rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_29287'))
(rdflib.term.URIRef('https://iconclass.org/46B332'), rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_29287'))
(rdflib.term.URIRef('https://iconclass.org/49E3911'), rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_29287'))
(rdflib.term.URIRef('https://iconclass.org/71I423'), rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_29287'))
(rdflib.term.URIRef('https://iconclass.org/94E231'), rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_29287'))
(rdflib.ter

Demonstrate doing a tiny query over the NFDI4Culture Graph

In [19]:
sparql_wrapper = SPARQLWrapper("https://nfdi4culture.de/sparql")
sparql_wrapper.setReturnFormat(JSON)
sparql_wrapper.setQuery("""
    PREFIX cto: <https://nfdi4culture.de/ontology#>
    SELECT *
    WHERE {
      ?resource cto:subjectConcept ?iconclass .
      OPTIONAL { ?resource rdfs:label ?resourceLabel . }
    }
    LIMIT 5
    """
)
print(sparql_wrapper.queryAndConvert())

{'head': {'link': [], 'vars': ['resource', 'iconclass', 'resourceLabel']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'resource': {'type': 'uri', 'value': 'http://www.bildindex.de/document/obj00000008'}, 'iconclass': {'type': 'uri', 'value': 'https://iconclass.org/25F62%28Herring%29'}, 'resourceLabel': {'type': 'literal', 'xml:lang': 'de', 'value': 'Lob des Herings'}}, {'resource': {'type': 'uri', 'value': 'http://www.bildindex.de/document/obj00000008'}, 'iconclass': {'type': 'uri', 'value': 'https://iconclass.org/41C6'}, 'resourceLabel': {'type': 'literal', 'xml:lang': 'de', 'value': 'Lob des Herings'}}, {'resource': {'type': 'uri', 'value': 'http://www.bildindex.de/document/obj00000008'}, 'iconclass': {'type': 'uri', 'value': 'https://iconclass.org/49MM32'}, 'resourceLabel': {'type': 'literal', 'xml:lang': 'de', 'value': 'Lob des Herings'}}, {'resource': {'type': 'uri', 'value': 'http://www.bildindex.de/document/obj00000008'}, 'iconclass': {'type': 'uri', 'value': 

In [28]:
sparql_wrapper = SPARQLWrapper("http://127.0.0.1:8000/")
sparql_wrapper.setReturnFormat(JSON)
sparql_wrapper.setQuery("""
    PREFIX cto: <https://nfdi4culture.de/ontology#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

    SELECT ?iconclass ?resource ?resourceLabel ?target
    WHERE {
        ?iconclass skos:relatedMatch ?target .
        
        SERVICE <https://nfdi4culture.de/sparql> {
            ?resource cto:subjectConcept ?iconclass .
            OPTIONAL { ?resource rdfs:label ?resourceLabel . }
        }
    }
    LIMIT 5
    """
)
print(sparql_wrapper.queryAndConvert())

HTTPError: HTTP Error 422: Unprocessable Entity

In [23]:




""" SERVICE <https://search.nfdi4chem.de/sparql> {} """
sparql = """\
PREFIX cto: <https://nfdi4culture.de/ontology#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>


SELECT *
WHERE {
    ?resource cto:subjectConcept ?iconclass .
    OPTIONAL { ?resource rdfs:label ?resourceLabel . }

    ?iconclass skos:relatedMatch ?target .

    SERVICE <https://nfdi4culture.de/sparql> {
      
    }
}
LIMIT 5
"""
import urllib.error
try:
    records = graph.query(sparql)
    click.echo(tabulate(records))
except urllib.error.HTTPError as e:
    print(type(e))
    print(e.url)
    print(e.code)
    print(e.headers)
    print(dir(e.fp)    )

<class 'urllib.error.HTTPError'>
https://nfdi4culture.de/sparql
500
Date: Wed, 17 Sep 2025 17:49:21 GMT
Server: Virtuoso/07.20.3240 (Linux) x86_64-ubuntu_focal-linux-gnu
Strict-Transport-Security: max-age=63072000; includeSubDomains; preload
Accept-Ranges: bytes
Content-Type: text/plain; charset=UTF-8
Content-Length: 2067
Access-Control-Allow-Origin: *
Access-Control-Allow-Methods: GET, OPTIONS
Access-Control-Allow-Headers: Content-Type, Authorization
Connection: close


['__abstractmethods__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_abc_impl', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_check_clos